# Контрольная работа
## Вариант 60
### Выполнил: студент ПИ19-3 Ковалев А.И.

## Задание 1

In [3]:
from functools import lru_cache

import numpy as np
np.set_printoptions(formatter={'float_kind': "{:.3f}".format})


def transition_probability(matrix, k):
    """Вероятность перехода из x в y за k шагов"""
    return np.linalg.matrix_power(matrix, k)


def state_probability(matrix, start, k):
    """Вероятность состояния за k шагов"""
    return start * np.linalg.matrix_power(matrix, k)


def first_transition_probability(matrix, k):
    """Вероятность первого перехода за k шагов"""
    size = len(matrix)
    prev, res = np.copy(matrix), np.zeros((size, size))

    range_ = range(size)
    for _ in range(k - 1):
        for i in range_:
            for j in range_:
                res[i, j] = sum(matrix[i, m] * prev[m, j] if m != j else 0 for m in range_)

        prev = np.copy(res)
        res = np.zeros((size, size))

    return prev


def last_transition_probability(matrix, k):
    """Вероятность перехода не позднее чем за k шагов"""
    return sum(first_transition_probability(matrix, t) for t in range(1, k + 1))


def avg_steps(matrix):
    """Среднее количество шагов, необходимых для первого перехода"""
    return sum(t * first_transition_probability(matrix, t) for t in range(1, 191))


def first_return_probability(matrix, k):
    """Вероятность первого возвращения на k-ом шаге"""
    _matrix = np.copy(matrix)
    p_jj = transition_probability

    @lru_cache(maxsize=None)
    def f_jj(_k):
        return p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])

    return np.diagonal(f_jj(k))


def avg_time_return(matrix):
    """Среднее время возвращения"""
    return sum(t * first_return_probability(matrix, t) for t in range(1, 191))


def last_return_probability(matrix, k):
    """Вероятность возвращения не позднее чем за k шагов"""
    return sum(first_return_probability(matrix, t) for t in range(1, k + 1))


def steady_state_probabilities(matrix):
    """Установившиеся вероятности"""
    matrix_ = np.copy(matrix).transpose()
    np.fill_diagonal(matrix_, np.diagonal(matrix_) - 1)
    matrix_[-1, :] = 1

    vec_b = np.zeros(len(matrix_))
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)


def validate(matrix):
    assert matrix.shape == (13, 13), "Размер матрицы должен быть (13, 13)"
    assert np.equal(np.sum(matrix, axis=1), np.matrix(np.ones(13))).all(), (
        "Сумма вероятностей в каждой строке должна равняться 1"
    )


def answer(res):
    return f"Ответ: {res}\n"

Система имеем 13 дискретных состояний. Изменение состояний происходит
в дискретные моменты времени с заданной вероятность. Схема марковского
процесса изображена на рисунке. Требуется определить:

In [4]:
transition_matrix = np.matrix([
    [0.09, 0.56, 0, 0, 0.34, 0.01, 0, 0, 0, 0, 0, 0, 0],
    [0.47, 0.3, 0, 0.18, 0.05, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0.31, 0.11, 0, 0.33, 0.25, 0, 0, 0, 0, 0, 0, 0],
    [0.24, 0.2, 0, 0.31, 0, 0, 0.25, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0.39, 0.61, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0.23, 0, 0.49, 0.28, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0.34, 0.24, 0, 0.04, 0, 0, 0, 0.35, 0.03, 0],
    [0, 0, 0, 0, 0.26, 0, 0.2, 0.5, 0, 0.02, 0.02, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0.07, 0.61, 0.32, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0.69, 0, 0.24, 0.07, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0.3, 0, 0, 0.31, 0.03, 0.13, 0.23],
    [0, 0, 0, 0, 0, 0, 0.59, 0.05, 0, 0, 0.29, 0.07, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.87, 0, 0.13]
])
validate(transition_matrix)
print("Матрица переходов:")
print(transition_matrix, "\n")

Матрица переходов:
[[0.090 0.560 0.000 0.000 0.340 0.010 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.470 0.300 0.000 0.180 0.050 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.000 0.310 0.110 0.000 0.330 0.250 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.240 0.200 0.000 0.310 0.000 0.000 0.250 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.000 0.000 0.000 0.390 0.610 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.000 0.000 0.230 0.000 0.490 0.280 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.000 0.000 0.000 0.340 0.240 0.000 0.040 0.000 0.000 0.000 0.350 0.030
  0.000]
 [0.000 0.000 0.000 0.000 0.260 0.000 0.200 0.500 0.000 0.020 0.020 0.000
  0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.070 0.610 0.320 0.000 0.000
  0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.690 0.000 0.240 0.070 0.000 0.000
  0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.300 0.000 0.000 0.310 0.030 0.130
  0.230]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.590 0.050 0.000 0.000 0.290 

In [5]:
k, i, j = 10, 10, 1
transition = transition_probability(transition_matrix, k)
print(f"1. вероятность того, что за {k} шагов система перейдет из состояния {i} в состояние {j}:")
print(answer(transition[i - 1, j - 1]))

1. вероятность того, что за 10 шагов система перейдет из состояния 10 в состояние 1:
Ответ: 0.1305648730673463



In [6]:
k = 10
a_0 = np.array([0, 0.03, 0.08, 0.14, 0.16, 0.04, 0.13, 0.11, 0.07, 0.11, 0.01, 0.06, 0.06])
state = state_probability(transition_matrix, a_0, k)
print(f"2. вероятности состояний системы спустя {k} шагов, если в начальный "
      f"момент вероятность состояний были следующими A={a_0}:")
print(answer(state))

2. вероятности состояний системы спустя 10 шагов, если в начальный момент вероятность состояний были следующими A=[0.000 0.030 0.080 0.140 0.160 0.040 0.130 0.110 0.070 0.110 0.010 0.060
 0.060]:
Ответ: [[0.146 0.176 0.001 0.218 0.217 0.002 0.104 0.005 0.020 0.027 0.057 0.012
  0.016]]



In [7]:
k, i, j = 9, 11, 5
first_transition = first_transition_probability(transition_matrix, k)
print(f"3. вероятность первого перехода за {k} шагов из состояния {i} в состояние {j}:")
print(answer(first_transition[i - 1, j - 1]))

3. вероятность первого перехода за 9 шагов из состояния 11 в состояние 5:
Ответ: 0.053530395066083813



In [8]:
k, i, j = 6, 6, 3
last_transition = last_transition_probability(transition_matrix, k)
print(f"4. вероятность перехода из состояния {i} в состояние {j} не позднее чем за {k} шагов:")
print(answer(last_transition[i - 1, j - 1]))

4. вероятность перехода из состояния 6 в состояние 3 не позднее чем за 6 шагов:
Ответ: 0.319602925188



In [9]:
i, j = 7, 10
avg_steps_number = avg_steps(transition_matrix)
print(f"5. среднее количество шагов для перехода из состояния {i} в состояние {j}:")
print(answer(avg_steps_number[i - 1, j - 1]))

5. среднее количество шагов для перехода из состояния 7 в состояние 10:
Ответ: 45.67463408023797



In [10]:
k, i = 7, 9
first_return = first_return_probability(transition_matrix, k)
print(f"6. вероятность первого возвращения в состояние {i} за {k} шагов:")
print(answer(first_return[i - 1]))

6. вероятность первого возвращения в состояние 9 за 7 шагов:
Ответ: 0.003200636206080007



In [11]:
k, i = 8, 12
last_return = last_return_probability(transition_matrix, k)
print(f"7. вероятность возвращения в состояние {i} не позднее чем за {k} шагов")
print(answer(last_return[i - 1]))

7. вероятность возвращения в состояние 12 не позднее чем за 8 шагов
Ответ: 0.2329925820293406



In [12]:
i = 7
avg_time_to_return = avg_time_return(transition_matrix)
print(f"8. среднее время возвращения в состояние {i}")
print(answer(avg_time_to_return[i - 1]))

8. среднее время возвращения в состояние 7
Ответ: 10.528699660009245



In [13]:
steady_state = steady_state_probabilities(transition_matrix)
print("9. установившиеся вероятности:")
print(answer(steady_state))

9. установившиеся вероятности:
Ответ: [0.157 0.189 0.001 0.223 0.225 0.002 0.095 0.003 0.013 0.021 0.049 0.010
 0.013]



## Задание 2

In [30]:
import numpy as np
np.set_printoptions(formatter={'float_kind': "{:.3f}".format})


def steady_state_probability(matrix):
    """Установившиеся вероятности"""
    size = len(matrix)
    matrix_ = np.copy(matrix).transpose()
    np.fill_diagonal(matrix_, [-sum(matrix_[:, i]) for i in range(size)])
    matrix_[-1, :] = 1

    vec_b = np.zeros(size)
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)


def failure_probability(s_state):
    """Вероятность отказа"""
    return s_state[-1]


def relative_and_absolute_service_intensity(s_state, la):
    """Относительная и абсолютная интенсивность обслуживания"""
    relative = 1 - s_state[-1]
    return relative, relative * la


def average_queue_length(s_state, m, n):
    """Средняя длина очереди"""
    return sum((i * s_state[m + i]) for i in range(1, n + 1))


def average_queue_time(s_state, m, n, mu):
    """Среднее время в очереди"""
    return sum(((i + 1) / (m * mu) * s_state[m + i]) for i in range(n))


def average_number_busy_channels(s_state, m, n):
    """Среднее число занятых каналов"""
    return (sum((i * s_state[i]) for i in range(1, m + 1)) +
            sum((m * s_state[i]) for i in range(m + 1, m + n + 1)))


def skip_queue_probability(s_state, m):
    """Вероятность не ждать в очереди"""
    return sum(s_state[:m])


def average_downtime(matrix):
    """Среднее время простоя системы массового обслуживания"""
    return 1 / np.sum(matrix, axis=1)


def init_matrix(n, m, la, mu):
    size = n + m + 1
    matrix = np.zeros((size, size))
    np.fill_diagonal(matrix[:, 1:], la)
    np.fill_diagonal(matrix[1:, :], [*[i * mu for i in range(1, m)], *[m * mu for _ in range(n + 1)]])
    return matrix


def validate(matrix):
    size = n + m + 1
    assert matrix.shape == (size, size), f"Размер матрицы должен быть {size}"


def answer(*args):
    res = ", ".join([str(i) for i in args])
    return f"Ответ: {res}\n"

Задана система массового обслуживания со следующими характеристиками:
- интенсивность поступления λ=32
- каналов обслуживания m=5
- интенсивность обслуживания μ=8
- максимальный размер очереди n=17

Изначально требований в системе нет.

In [31]:
la = 32
m = 5
mu = 8
n = 17
initial_matrix = init_matrix(n, m, la, mu)
validate(initial_matrix)
initial_matrix

array([[0.000, 32.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000],
       [8.000, 0.000, 32.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 16.000, 0.000, 32.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 0.000, 24.000, 0.000, 32.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 0.000, 0.000, 32.000, 0.000, 32.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 0.000, 0.000, 0.000, 40.000, 0.000, 32.000, 0.0

In [32]:
steady_state = steady_state_probability(initial_matrix)
print("a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и "
      "найдите установившиеся вероятности состояний:")
print(answer(steady_state))

a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и найдите установившиеся вероятности состояний:
Ответ: [0.013 0.052 0.105 0.140 0.140 0.112 0.090 0.072 0.057 0.046 0.037 0.029
 0.023 0.019 0.015 0.012 0.010 0.008 0.006 0.005 0.004 0.003 0.003]



In [33]:
fail_probability = failure_probability(steady_state)
print("b) Найдите вероятность отказа в обслуживании:")
print(answer(fail_probability))

b) Найдите вероятность отказа в обслуживании:
Ответ: 0.0025206623548613455



In [34]:
relative, absolute = relative_and_absolute_service_intensity(steady_state, la)
print("c) Найдите относительную и абсолютную интенсивность обслуживания:")
print(answer(relative, absolute))

c) Найдите относительную и абсолютную интенсивность обслуживания:
Ответ: 0.9974793376451386, 31.919338804644436



In [35]:
avg_queue_len = average_queue_length(steady_state, m, n)
print("d) Найдите среднюю длину в очереди:")
print(answer(avg_queue_len))

d) Найдите среднюю длину в очереди:
Ответ: 2.016979619710541



In [36]:
avg_queue_time = average_queue_time(steady_state, m, n, mu)
print("e) Найдите среднее время в очереди:")
print(answer(avg_queue_time))

e) Найдите среднее время в очереди:
Ответ: 0.06303061311595441



In [37]:
avg_number_channels = average_number_busy_channels(steady_state, m, n)
print("f) Найдите среднее число занятых каналов:")
print(answer(avg_number_channels))

f) Найдите среднее число занятых каналов:
Ответ: 3.989917350580555



In [38]:
skip_probability = skip_queue_probability(steady_state, m)
print("g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди:")
print(answer(skip_probability))

g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди:
Ответ: 0.4503831726848609



In [39]:
avg_downtime = average_downtime(initial_matrix)
print("h) Найти среднее время простоя системы массового обслуживания:")
print(avg_downtime)

h) Найти среднее время простоя системы массового обслуживания:
[0.031 0.025 0.021 0.018 0.016 0.014 0.014 0.014 0.014 0.014 0.014 0.014
 0.014 0.014 0.014 0.014 0.014 0.014 0.014 0.014 0.014 0.014 0.025]
